In [1]:
import pandas as pd
import pickle
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import json
import numpy as np
from numpy import cov
import csv

## Формирование фрейма для анализа алгоритма

In [20]:
df = pd.read_excel('INITIAL_DATA/Свод_Почасовая выработка 2019 и 2020 обновл.xls', sheet_name=None, usecols="A:E", skiprows=1,nrows=750, header = None, names = ['timestamp','Solar_generation','External_grid','Total_consumption', 'PVSyst' ])


In [22]:
df.dtypes

AttributeError: 'dict' object has no attribute 'dtypes'

In [23]:
df.tail(10)

AttributeError: 'dict' object has no attribute 'tail'

In [24]:
cdf = pd.concat(df,axis=0, join="inner", ignore_index=True, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

In [30]:
cdf.tail(1950)

,timestamp,Solar_generation,External_grid,Total_consumption,PVSyst
14243,2020-12-01 23:00:00,0,85.3918,85.3918,0
14244,2020-12-02 00:00:00,0,85.9525,85.9525,0
14245,2020-12-02 01:00:00,0,77.572,77.572,0
14246,2020-12-02 02:00:00,0,85.7012,85.7012,0
14247,2020-12-02 03:00:00,0,86.4451,86.4451,0
...,...,...,...,...,...
16188,NaN,NaN,NaN,NaN,NaN
16189,NaN,NaN,NaN,NaN,NaN
16190,NaN,NaN,NaN,NaN,NaN
16191,NaN,NaN,NaN,NaN,NaN
